# **Deep Learning Models**
   Goal: Capture complex temporal patterns
## Models
 - LSTM
 - GRU
 - Temporal CNN
 - Seq2Seq
 - Transformer (advanced)

## Techniques
 - Sliding windows
 - Multivariate sequences
 - Early stopping
 - GPU acceleration (if available)

  Output: Best DL model


# DL Foundations & Data Preparation (CRITICAL)
## Load Feature-Engineered Dataset


In [1]:
# Clone GitHub Repository
!git clone https://github.com/sabin74/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform.git

Cloning into 'Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 216 (delta 111), reused 65 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (216/216), 27.42 MiB | 39.16 MiB/s, done.
Resolving deltas: 100% (111/111), done.
Filtering content: 100% (18/18), 348.57 MiB | 83.18 MiB/s, done.


In [3]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Set Project Root
os.chdir('/content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform')
print("Current Directory: ", os.getcwd())

Current Directory:  /content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform


In [7]:
# Load Feature Engineered Data
df = pd.read_parquet('data/features/train_features.parquet')

df['data'] = pd.to_datetime(df['date'])
df = df.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,...,is_pre_holiday,is_post_holiday,family_freq,store_freq,city_freq,state_freq,sales_log,family_te,store_te,data
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0,0,0.030303,0.018519,0.333333,0.351852,0.000000,1.612195,3.148359,2013-01-01
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,...,0,1,0.030303,0.018519,0.333333,0.351852,1.098612,1.612195,3.148359,2013-01-02
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,0,0,0.030303,0.018519,0.333333,0.351852,1.386294,1.612195,3.148359,2013-01-03
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,0,0,0.030303,0.018519,0.333333,0.351852,1.386294,1.612195,3.148359,2013-01-04
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13,...,0,0,0.030303,0.018519,0.333333,0.351852,1.791759,1.612195,3.148359,2013-01-05


In [11]:
# Train Validatoin Split
split_date = '2017-01-01'
train_df = df[df['data'] < split_date].copy()
valid_df = df[df['data'] >= split_date].copy()

print(df.shape, train_df.shape, valid_df.shape)

(3054348, 63) (2642706, 63) (411642, 63)


In [28]:
# Feature Selection
TARGET = "sales"

NUMERIC_FEATURES = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
NUMERIC_FEATURES.remove(TARGET)

CATEGORICAL_FEATURES = df.select_dtypes(include='category').columns.tolist()

In [30]:
from pandas.core.indexes import category
# Encode Categoricals (Integer IDs for Embeddings)
# Build Category
category_maps = {}

for col in CATEGORICAL_FEATURES:
    category_maps[col] = {
        category: idx
        for idx, category in enumerate(train_df[col].astype(str).unique())
    }

In [31]:
# Apply Encoding
def encode_categoreis(df, category_maps):
  df = df.copy()
  for col, mapping in category_maps.items():
    df[col] = df[col].astype(str).map(mapping).fillna(-1).astype(int)
  return df

train_df = encode_categoreis(train_df, category_maps)
valid_df = encode_categoreis(valid_df, category_maps)

In [32]:
# Normalize Numeric Features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df[NUMERIC_FEATURES] = scaler.fit_transform(train_df[NUMERIC_FEATURES])
valid_df[NUMERIC_FEATURES] = scaler.transform(valid_df[NUMERIC_FEATURES])

In [37]:
# Create Window Sequence

,store_nbr
0,-1.700267
1,-1.700267
2,-1.700267
3,-1.700267
4,-1.700267
...,...
3054112,1.700267
3054113,1.700267
3054114,1.700267
3054115,1.700267
